# Actividad 03 Reglas de Asociación

Autor     : Jorge Andre Salcedo Hurtado

Proposito : Reglas de Asociación

Problema  : 

* Implementar el algoritmo Apriori
* Aplicar el algoritmo y obtener reglas de asociación
* Explicar las reglas obtenidas


# Librerias Utilizadas

In [2]:
import pandas as pd
import numpy as np
import itertools
#libreria utilizada para paralelizar procesos utilizando los nucleos del procesador
from multiprocessing.pool import ThreadPool 
import os

# Leemos los datos


In [179]:
#Utilizamos la funcion np.load para leer el dataset brindado, convirtiendolo a una lista
playlists = np.load('../DATASETS/spotify.npy', allow_pickle=True).tolist()
#Extraemos las playlist de nuestro dataSet con el metodo values()
playlists = playlists.values()

# 1. Funciones(Implementar el algoritmo Apriori)

## 1.1 Funcion para obetener el soporte de un itemset

In [180]:
def getSupport(itemset):
    """Funcion para obtener el soporte del itemset brindado"""
    #Inicializamos nuestro soporte en 0
    support = 0
    #Recorremos cada playlist en busca de nuestro itemset
    for sample in playlists:
        #Obtenemos una interseccion entre el itemset y el playlist analizado
        intersection = [elem for elem in itemset if elem in sample]
        #En caso sean iguales, aniadimos una aparicion en el contador
        if(len(intersection) == len(itemset)):
            support +=1
    return itemset,support

## 1.2 Funcion para obtener conjuntos de elementos frecuentes

In [223]:
def get_frequent_itemsets(playlists, min_support):
    """Recibe la esctructura dedatos que contiene a las playlists y 
    retorna una estructura con los itemsets frecuentes, bajo un 
    umbral mínimo de confianza."""
    # ------------------------------------ PARA 1-ITEMSETS ------------------------------------
    #inicializamos nuestro conjunto de canciones vacio
    songs = []
    # recorremos todas las playlists
    for sample in playlists:
        # añadimos todas las canciones en la playlist actual a nuestro total de canciones
        songs+=list(set(sample))
    # usando la funcion pd.value_counts(), le pasamos nuestra lista de canciones y nos devuelve una tabla de frecuencias 
    df = pd.value_counts(songs).to_frame().reset_index()
    # aniadimos los nombres a las columnas
    df.columns = ['Col_value','Count']
    # dividimos los valores de todos los soportes entre el total de playlists
    df.Count/=len(playlists)
    # seleccionamos solo aquellas canciones cuyo support sea mayor a nuestro min_support
    df1 = df[df.Count>=min_support]
    # exportamos 
    df1.to_csv('1_itemset.csv')

    # ------------------------------------ PARA 2-ITEMSETS ------------------------------------
    # recuperamos todos nuestros 1-itemsets que superan el umbral
    itemsets_1 = df1.Col_value.tolist()
    # ordenamos nuestra lista
    itemsets_1.sort()
    # declaramos un arreglo donde irán todos nuestros posibles 2-itemsets
    itemsets_2 = []
    # realizamos una combinacion de todos los 1-itemsets de 2 en 2
    itemsets_2.extend(list(itertools.combinations(itemsets_1,r=2)))
    # creamos nuestro objeto ThreadPool pasandole como parametro la cantidad de nucleos de nuestro computador
    pool = ThreadPool(os.cpu_count())
    # mapeamos el proceso requerido a los nucleos, pasandole la funcion de obtener soporte y nuestra lista iterable de 2-itemsets
    itemsets_22 = pool.map(getSupport,itemsets_2)
    # creamos una tabla auxiliar para almacenar todos los 2-itemsets con sus soportes obtenidos
    dfaux = pd.DataFrame(itemsets_22, columns=['Col_value','Count'])
    # dividimos los soportes entre el total de playlists del a BD
    dfaux.Count/=len(playlists)
    # elegimos solo aquellos 2-itemsets que pasan el umbral
    dfaux = dfaux[dfaux.Count>=min_support]

    # ------------------------------------ PARA k-ITEMSETS ------------------------------------
    # creamos un arreglo de dataframes que almacenara todos los dataframes de N-itemsets
    df_itemsets = [df1,dfaux.reset_index(drop=True)]
    # declaramos la variable que nos ayudara a recordar cual fue el numero de items anterior
    k = 3
    while True:
        # recuperamos los valores de los k-itemsets anteriores como una lista
        restItemSets = dfaux.Col_value.tolist()
        # creamos un arreglo para almacenar el nuevo conjunto de k-itemsets
        newItemSets = []
        # declaramos un indice para recorrer los itemsets
        index = 1
        # recorremos el itemset para usarlo como primer elemento a comparar
        for _setA in restItemSets:
            # recorremos el itemset para usarlo como segundo elemento a comparar desde un indice siguiente al que empieza el anterior bucle
            for _setB in restItemSets[index:]:
                # si ambos itemsets se parecen en sus k-2 primeros items 
                if _setA[:k-2]==_setB[:k-2]:
                    # en caso sean similares, aniadimos el nuevo itemset a nuestro arreglo
                    newItemSets.append(list(set(_setA+_setB)))
            # incrementamos nuestro indice para que no se repitan valores
            index+=1
        # en caso que nuestro arreglo de nuevos posibles k-itemsets este vacio, terminara el algoritmo
        if(len(newItemSets)==0):
            break
        # en caso contrario, obtendremos los soportes de los nuevos k-itemsets
        newItemSets = pool.map(getSupport,newItemSets)
        # actualizamos nuestro dataFrame con los nuevos k-itemsets
        dfaux = pd.DataFrame(newItemSets, columns=['Col_value','Count'])
        # dividimos su valor de soporte obtenido entre el total de playlists de la BD
        dfaux.Count/=len(playlists)
        # filtramos solo aquellos k-itemsets que pasan el umbral
        dfaux = dfaux[dfaux.Count>=min_support]
        # en caso si exista alguno que pase el umbral
        if (not dfaux.empty):
            # aniadimos nuestro nuevo dataframe de k-itemsets a nuestro arreglo total
            df_itemsets.append(dfaux.reset_index(drop=True))
        # incrementamos k para continuar con el siguiente k-itemset
        k+=1
    # retornamos el arreglo de dataFrames que contiene nuestros k-itemsets
    return df_itemsets

## 1.3 Funcion para generar reglas de asociación 

In [271]:
def generate_association_rules(frequent_itemsets, confidence = 0, lift = 0):
    """Recibe los itemsets frecuentes generados por la función anterior y retorna las
    reglas de asociación. Se le puede entregar umbrales de confianza o lift para las
    reglas que se retornarán. Por ejemplo, si se llama esta función con los argumentos
    confidence = 0.5 y lift = 1.2, se espera que se retornen aquellas reglas que
    cumplan con una confianza ≥ 0.5 y un lift ≥ 1.2."""
    # declaramos un arreglo para almacenar las reglas obtenidas
    association_rules = []
    # recorremos cada dataFrame anteriormente obenido de k-itemsets, para >=2
    for df in frequent_itemsets[1:]:
        # recorremos cada itemset del dataframe actual
        for itemset in df['Col_value']:
            # recuperamos el soporte de nuestro itemset actual
            general_support = getSupport(itemset)[1]
            # creamos un arreglo vacio de antecedentes
            antecedents=[]
            # de forma iterativa, creamos todas las posibles combinaciones que se puede hacer
            # con nuestro actual itemset, para luego utilizar esto como posibles antecedentes
            for i in range(1,len(itemset)):
                antecedents+= list(itertools.combinations(iterable=itemset,r=i))
            # recorremos los antecedentes(x)
            for X in antecedents:
                # obtenemos el soporte de nuestro actual antecedente
                antecedent_support = getSupport(X)[1]
                # obtenemos y_x, hallando la diferencia del itemset y X
                y_x = list(set(itemset).difference(set(X)))
                # obtenemos la confiaza de nuestra regla actual
                current_confidence = general_support/antecedent_support
                # obtenemos el lift de nuestra regla actual
                current_lift = current_confidence/(getSupport(y_x)[1]/len(playlists))
                # si nuestra actual confianza y lift superan el umbral
                if(current_confidence>=confidence and current_lift>=lift):
                    # aniadimos la actual regla a nuestro arreglo
                    association_rules.append([X,y_x,general_support,current_confidence,current_lift])
    # devolvemos aquellas reglas que hayan superado el umbral
    return association_rules

# 2. Aplicar el algoritmo y obtener reglas de asociación

## 2.1 Ejecucion del algoritmo para obtener los itemsets frecuentes
Para este problema utilizamos como soporte minimo 1.5%, que al tener un dataset tan grande de 10000 muestras, nos indica que los itemsets deberian aparecer 150 veces o mas para ser considerados, siendo 150 un numero de apariciones considerable para encontrar algun patrón.


In [256]:
# llamamos a nuestra funcion utilizando como soporte minimo 0.015
frequent_itemsets = get_frequent_itemsets(playlists=playlists, min_support=0.015)

### 2.1.1 1-itemsets

In [267]:
display(frequent_itemsets[0])

,Col_value,Count
0,Closer,0.0723
1,HUMBLE.,0.0465
2,Home,0.0454
3,Roses,0.0424
4,Ride,0.0414
...,...,...
344,7/11,0.0150
345,The Ocean,0.0150
346,Come and See Me (feat. Drake),0.0150
347,Ms. Jackson,0.0150


### 2.1.2 2-itemsets

In [268]:
display(frequent_itemsets[1])

,Col_value,Count
0,"(Bad and Boujee (feat. Lil Uzi Vert), Bounce B...",0.0169
1,"(Bad and Boujee (feat. Lil Uzi Vert), Broccoli...",0.0155
2,"(Bad and Boujee (feat. Lil Uzi Vert), Caroline)",0.0153
3,"(Bad and Boujee (feat. Lil Uzi Vert), HUMBLE.)",0.0167
4,"(Bad and Boujee (feat. Lil Uzi Vert), Mask Off)",0.0151
5,"(Bad and Boujee (feat. Lil Uzi Vert), iSpy (fe...",0.0152
6,"(Bounce Back, HUMBLE.)",0.0156
7,"(Broccoli (feat. Lil Yachty), Caroline)",0.0172
8,"(Broccoli (feat. Lil Yachty), No Problem (feat...",0.0158
9,"(Closer, Let Me Love You)",0.0159


## 2.2 Ejecucion del algoritmo para obtener las reglas de asociacion
Para este ejemplo consideramos:
* Una confianza del 50%, que indica que por lo menos nuestra regla deberia aparecer en la mitad del total de apariciones del antecedente. Es decir, del total de personas que escuchan un conjunto de X canciones, será una regla confiable si al menos la mitad de estas personas escuchan el conjunto de canciones Y-X.
* Elegimos un valor de lift mayor a 1, lo cual nos indicará que ese conjunto aparece una cantidad de veces superior a lo esperado bajo condiciones de independencia (por lo que se puede intuir que existe una relación que hace que los productos se encuentren en el conjunto más veces de lo normal).


In [272]:
# llamamos a nuestra funcion utilizando como confianza minima de 0.5 y lift 1.2
association_rules = generate_association_rules(frequent_itemsets, 0.5, 1.2)

### 2.2.1 Guardamos nuestras reglas en un dataframe y generamos su csv

In [273]:
# convertimos a dataframe nuestras reglas obtenidas
association_rulesdf = pd.DataFrame(association_rules, columns=['X','Y','Support','Confidence','Lift'])
# guardamos como csv el resultado
association_rulesdf.to_csv('ReglasAsociacion.csv')

## 2.3 Mejores 10 reglas de asociación

In [279]:
association_rulesdf = association_rulesdf.sort_values(by=['Confidence','Lift','Support'],ascending = False).reset_index(drop = True)
association_rulesdf.head(10)

,X,Y,Support,Confidence,Lift
0,"(DNA.,)",[HUMBLE.],190,0.822511,17.688405
1,"(Mask Off,)",[HUMBLE.],204,0.645570,13.883218
2,"(XO TOUR Llif3,)",[HUMBLE.],204,0.631579,13.582343
3,"(Bounce Back,)",[Bad and Boujee (feat. Lil Uzi Vert)],169,0.569024,16.493437
4,"(XO TOUR Llif3,)",[Congratulations],179,0.554180,13.751354
5,"(goosebumps,)",[HUMBLE.],167,0.542208,11.660383
6,"(Congratulations,)",[HUMBLE.],214,0.531017,11.419728
7,"(Bounce Back,)",[HUMBLE.],156,0.525253,11.295753
8,"(Mask Off,)",[XO TOUR Llif3],163,0.515823,15.969746
9,"(Mask Off,)",[Congratulations],162,0.512658,12.721048


# 3. Explicar las reglas obtenidas
Deberás seleccionar 4 reglas y comentar su calidad de acuerdo a los diferentes indicadores disponibles (support, confidence y lift). Además, según el género y/o el artista de las canciones (que puedes buscar según el nombre de la canción) deberás darle una
breve interpretación a las reglas.

## 3.1 {DNA} => {HUMBLE}
Esta regla cuenta con:
* El valor mas alto de CONFIDENCE entre las otras.
* A su vez, supera por 40 apariciones nuestro valor de SUPPORT minimo que era de 150. 
* Y por ultimo tambien cuenta con el valor LIFT mas elevado, lo cual indica que esta regla no se debe al azar, sino que representa una regla real.

Finalmente demostrando que es lo mas cercano que tenemos a una regla real entre todas

In [289]:
association_rulesdf[0:1]

,X,Y,Support,Confidence,Lift
0,"(DNA.,)",[HUMBLE.],190,0.822511,17.688405


## 3.2 {Congratulations} => {HUMBLE}
Esta regla nos muestra:
* Un valor de soporte de 214, el valor mas elevado entre todos que indica una mayor aparicion.
* Una confianza de 0.531017 un poco cercana a nuestro valor minimo, haciendo que pueda parecer una regla algo aleatoria.
* Pero finalmente cuenta con un lift de 11.419728 que una vez mas nos indica que si estamos frente a un patron y no un suceso aleatorio.


In [288]:
association_rulesdf[6:7]

,X,Y,Support,Confidence,Lift
6,"(Congratulations,)",[HUMBLE.],214,0.531017,11.419728


## 3.3 {XO TOUR Llif3} => {Mask Off}
Esta regla presenta:
* Un soporte de 163 muy cercano a nuestro minimo, indicando la menor cantidad de apariciones.
* La confianza mas baja de nuestro conjunto (0.504644), llegando a aparecer la regla mas de la mitad de veces que su antecedente por casi nada.
* Un lift muy alejado de 1

Pero, por los dos primeros parametros y la tan baja confianza que muestra, podemos decir que puede ser un suceso aleatorio.

In [290]:
association_rulesdf[10:11]

,X,Y,Support,Confidence,Lift
10,"(XO TOUR Llif3,)",[Mask Off],163,0.504644,15.969746


## 3.4 {Bounce Back} => {HUMBLE}
Finalmente escogimos la regla que tiene:
* El valor lift 11.29 mas bajo entre nuestras reglas
* Una confianza cercana a nuestra confianza minima
* Y un valor de soporte con tan solo 6 apariciones por encima del valor minimo.

Por lo que llegamos a la conclusion de que esta regla es lo mas cercano a un hecho aleatorio y lo menos parecido a una regla entre todas.

In [292]:
association_rulesdf[7:8]

,X,Y,Support,Confidence,Lift
7,"(Bounce Back,)",[HUMBLE.],156,0.525253,11.295753


## 3.5 Apreciacion general
En los playlists habia canciones de todo tipo de genero; pero, al tener en las reglas puras canciones RAP de artistas afroamericanos, se puede apreciar que estas si guardan una coherencia aceptable. Como en la primera regla considerada {DNA} => {HUMBLE}, donde ambas canciones del artista Kendrick Lamar nos indica que las personas que agregaron la primera cancion DNA tambien les gustó y aniadieron HUMBLE a sus playlists. Así como tambien se puede apreciar que esta última cancion HUMBLE es la consecuencia de 6 del total de 11 reglas encontradas.